In [1]:
import os

In [8]:
os.chdir('../')

In [9]:
%pwd

'e:\\Projects\\text-Summarization'

In [19]:
from dataclasses import dataclass
from pathlib import Path    

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [20]:
from textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummarizer.utils.common import read_yaml, create_dirs

In [21]:
class ConfigurationManager:
    def __init__(
            self, 
            config_file_path = CONFIG_FILE_PATH,
            params_file_path =  PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_dirs([self.config.artifacts_root])

    def data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_dirs([config.root_dir])
        
        return DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

In [22]:
import os
import zipfile
import urllib.request as request
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [23]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            url, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded file from {url} to {self.config.local_data_file}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")

    def unzip_file(self):
        uzip_path = self.config.unzip_dir
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(uzip_path)   

In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    print(e)

[2024-09-14 02:09:34,899: INFO: common: Read yaml file from config\config.yaml]
[2024-09-14 02:09:34,902: INFO: common: Read yaml file from params.yaml]
[2024-09-14 02:09:34,905: INFO: common: Created directory artifacts]
[2024-09-14 02:09:34,907: INFO: common: Created directory artifacts/data_ingestion]
[2024-09-14 02:09:40,365: INFO: 3045281734: Downloaded file from artifacts/data_ingestion/data.zip to artifacts/data_ingestion/data.zip]
